In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import string
import matplotlib.gridspec as gridspec
from scipy import stats
from scipy.stats import gaussian_kde
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [ ]:
plt.style.use('bmh')
plt.rcParams.update({'font.size': 7})
plt.rcParams.update({"font.family": "DejaVu Sans"})

In [ ]:
# load Rrs - IOP dataset
ds = xr.load_dataset('./data/psc_rrs_iop.nc')
# load inversion results
ds_inv = xr.load_dataset('./data/psc_rrs_inversion.nc')

In [ ]:
ds_inv['iop'] = ['a', 'bb']

In [ ]:
ds_inv['r_error'] = ds_inv.std_error/ds_inv.conc

In [ ]:
ds_inv['conc'] = ds_inv.conc.where(ds_inv.r_error<2)
ds_inv['iops'] = ds_inv.iops.where(ds_inv.r_error<2)

In [ ]:
# ds_inv['conc'] = ds_inv.conc.where(id_nan)
#ds_inv['iops'] = ds_inv.iops.where(id_nan)

In [ ]:
def mean_normalised_bias(x, y):
    return np.sum((y-x)/x)/len(x)

def mean_bias(x, y):
    return np.sum(y-x)/len(x)

In [ ]:
def plot_validation(xy, x_label=None, y_label=None, ncol=2, **kwargs):
    nrow = int(np.ceil(len(xy)/ncol))
    fig, axs = plt.subplots(nrow, ncol, figsize=kwargs.get('figsize'), dpi=150)
    axs = axs.flat
    # font
    font = {'fontname': kwargs.get('font')}
    
    for (i,j) in zip(xy, list(string.ascii_lowercase)):
        x = i.get('x').values
        y = i.get('y').values
        # total number of observation
        n = len(x)
        # mask nans
        mask = ~np.any([np.isnan(y), y==0], axis=0)
        x = x[mask]
        y = y[mask]
        # calculate scatter point densities
        xy_log = np.vstack([np.log10(x),np.log10(y)])
        z = gaussian_kde(xy_log)(xy_log)
        # Sort the points by density, so that the densest points are plotted last
        idx = z.argsort()
        x, y, z = x[idx], y[idx], z[idx]
        
        ax = next(axs)
        # determine min and max of axes
        ax_min, ax_max = [np.min(x)*.2, np.max(x)*2.5]
        ay_min, ay_max = [np.min(y)*.2, np.max(y)*2.5]
        # log-transform x and y
        x_log = np.log10(np.reshape(x, (-1,1)))
        y_log = np.log10(np.reshape(y, (-1,1)))
        # calculate regression stats
        lm = LinearRegression().fit(x_log,y_log)
        # regression line
        x_log_range = np.logspace(np.log10(ax_min), np.log10(ax_max))
        lm_line = np.power(10, lm.coef_[0][0]*np.log10(x_log_range)+lm.intercept_[0])
        # color
#         col = np.log(bbdm_550)[mask]
        
        # plot scatter
        ax.plot(x_log_range,  lm_line, color='red', linewidth=.5, linestyle='--')
        #ax.scatter(x[mask], y[mask], c=col[mask], alpha=.7, edgecolor='black')
        ax.scatter(x, y, alpha=.7, c=z, s=25, edgecolor='black', linewidth=.05)
        ax.plot(np.linspace(ax_min, ax_max),
                np.linspace(ax_min, ax_max), 'k', linewidth=.5)
        # stats
        mae = np.power(10, mean_absolute_error(x_log, y_log))
        mb = np.power(10, mean_bias(x_log, y_log))
        r2 = r2_score(x_log,y_log)
        slope = lm.coef_[0][0]
        intercept = lm.intercept_[0]
        
        # plot stats
        stat_str = 'MAE: {0:.3g}\nBias: {1: .3g}\nR2: {2:.3g}\nSlope: {3:.3g}\nIntercept: {4:.3g}\nf: {5:.3g}'\
                .format(mae,
                        mb,
                        r2,
                        slope,
                        intercept,
                        len(x)/n)
        ax.annotate(stat_str, xy=(.05, .95), xycoords='axes fraction', ha='left', va='top')
        ax.set_title('({})'.format(j), loc='left')
        # set axis labels and scale
        #ax.set_xlabel('forward modelled ({})'.format(i.get('units')), **font)
        #ax.set_ylabel('inverse modelled ({})'.format(i.get('units')), **font)
        ax.set_xscale('log')
        ax.set_yscale('log')
        # set axis limits
        ax.set_xlim([ax_min, ax_max])
        ax.set_ylim([ay_min, ay_max])  
   
    [i.set_xlabel('forward modelled ($mg\ m^{-3}$)') for i in axs[-2:]]
    [i.set_ylabel('inverse modelled ($mg\ m^{-3}$)') for i in axs[0:4:2]]
    # optional: delete last subplot
    #fig.delaxes(axs[3])
    fig.subplots_adjust(right=0.8)
    fig.tight_layout()
    
    
    return None

### Chlorophyll retrievals

In [ ]:
# pico
pico = {
    'x': ds.conc.sel(comp='pico'),
    'y': ds_inv.conc.sel(comp='pico'),
    'units': '$mg\ m^{-3}$',
    'name': 'pico'
}
# nano
nano = {
    'x': ds.conc.sel(comp='nano'),
    'y': ds_inv.conc.sel(comp='nano'),
    'units': '$mg\ m^{-3}$',
    'name': 'nano'
}
# micro
micro = {
    'x': ds.conc.sel(comp='micro'),
    'y': ds_inv.conc.sel(comp='micro'),
    'units': '$mg\ m^{-3}$',
    'name': 'micro'
}
# total chlorophyll
chl = {
    'x': ds.conc.sel(comp=['pico', 'nano', 'micro']).sum(axis=1),
    'y': ds_inv.conc.sel(comp=['pico', 'nano', 'micro']).sum(axis=1),
    'units': '$mg\ m^{-3}$',
    'name': 'chl'
}

In [ ]:
plot_validation([pico, nano, micro, chl], figsize=(6,6))
#plt.savefig('psc_retrieval.svg', format='svg')

### IOP retrievals

Absorption by detrital matter and CDOM...

In [ ]:
# detrital matter absorption
adm_440 = {
    'x': ds.iops.sel(comp='dm', iop='a', wavelength=440),
    'y': ds_inv.iops.sel(comp='dm', iop='a', wavelength=440),
    'units': '$m^{-1}$',
    'name': 'detrital matter $a(440)$'
}
# detrital matter bb
bbdm_550 = {
    'x': ds.iops.sel(comp='dm', iop='bb', wavelength=550),
    'y': ds_inv.iops.sel(comp='dm', iop='bb', wavelength=550),
    'units': '$m^{-1}$',
    'name': 'detrital matter $b_b(550)$'
}
# cdom
cdom_440 = {
    'x': ds.iops.sel(comp='cdom', iop='a', wavelength=440),
    'y': ds_inv.iops.sel(comp='cdom', iop='a', wavelength=440),
    'units': '$m^{-1}$',
    'name': 'cdom $a(440)$'
}

In [ ]:
plot_validation([adm_440, bbdm_550, cdom_440], figsize=(6,6))
#plt.savefig('cdom_dm_iop_retrieval.svg', format='svg')

Total absorption (at 440 nm) and backscatter (at 550 nm)...

In [ ]:
# total absorption @ 440 nm
a_440 = {
    'x': ds.iops.sel(wavelength=440, iop='a').sum(axis=1),
    'y': ds_inv.iops.sel(wavelength=440, iop='a').sum(axis=1),
    'units': '$m^{-1}$',
    'name': 'total absorption (440 nm)'
}
# total backscatter @ 550 nm
bb_550 = {
    'x': ds.iops.sel(wavelength=550, iop='bb').sum(axis=1),
    'y': ds_inv.iops.sel(wavelength=550, iop='bb').sum(axis=1),
    'units': '$m^{-1}$',
    'name': 'total backscatter (550 nm)'
}

In [ ]:
plot_validation([a_440, bb_550], figsize=(12,6))

### $R_{rs}$ validation

In [ ]:
#rrs @ 440
rrs_440 = {
    'x': ds.rrs.sel(wavelength=440),
    'y': ds_inv.rrs.sel(wavelength=440),
    'units': '$sr^{-1}$',
    'name': '$R_{rs}(440)$'
}
# rrs @ 550
rrs_550 = {
    'x': ds.rrs.sel(wavelength=550),
    'y': ds_inv.rrs.sel(wavelength=550),
    'units': '$sr^{-1}$',
    'name': '$R_{rs}(550)$'
}

In [ ]:
plot_validation([rrs_440, rrs_550], figsize=(12,6))